In [1]:
import pandas as pd
import spacy
nlp = spacy.load('es_core_news_lg')
from sentence_transformers import SentenceTransformer, util

c:\Users\a.obregon\.conda\envs\kMeans\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Función para cargar el texto desde un archivo con manejo de errores
def load_text_from_file(filename, encodings=None):
    if encodings is None:
        # Lista de codificaciones a probar
        encodings = ['utf-8', 'latin-1', 'latin1', 'ISO-8859-1']
    for encoding in encodings:
        try:
            with open(filename, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            pass
    raise Exception(
        "No se pudo decodificar el archivo con ninguna codificación.")

In [3]:
def process_sections_text(sections_text):
    sections = {}
    sections_text1 = sections_text.split("\n\n")
    for section_text in sections_text1:
        lines = section_text.split('\n')
        section_name = lines[0]
        section_text = '\n'.join(lines[1:])
        sections[section_name] = section_text
    return sections

In [4]:
def tokenize_sections(sections, nlp_model):
    section_tokenizer = {}
    for section_name, section_content in sections.items():
        section_doc = nlp_model(section_content)
        # section_tokens = [token for token in section_doc]
        section_tokenizer[section_name] = section_doc
    return section_tokenizer

In [ ]:
# def process_query(query, tokenized_sections, nlp_model):
#     nlp_query = nlp_model(query)
#     relevant_sections = None
#     for section in tokenized_sections:

In [5]:
file_name = 'crudArticulo.txt'

In [6]:
# Carga el texto desde el archivo con manejo de errores
try:
    text = load_text_from_file(file_name)
except Exception as e:
    print("Error:", e)
    exit(1)

In [7]:
sections = process_sections_text(text)
section_tokenizer = tokenize_sections(sections, nlp)

In [8]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Carga un modelo de embeddings
# embedder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')  # Carga un modelo de embeddings

In [16]:
# questions = ["Como realizar una gestión de la evolución y operativa de los artículos"]
questions = "¿Qué son los artículos?"
query_embedding = embedder.encode(questions, convert_to_tensor=True)

In [10]:
# Dividir en oraciones utilizando SpaCy
sentences = nlp(text).sents  
sentence_embeddings = []
for sentence in sentences:
    sentence_embedding = embedder.encode(sentence.text, convert_to_tensor=True)
    sentence_embeddings.append((sentence.text, sentence_embedding))

In [11]:
# Calcular la similitud con las oraciones y almacenar en una lista
similarity_list = []
for sentence_text, sentence_embedding in sentence_embeddings:
    similarity = util.pytorch_cos_sim(query_embedding, sentence_embedding)[0][0].item()
    similarity_list.append((sentence_text, similarity))

# Ordenar la lista por similitud en orden descendente
similarity_list.sort(key=lambda x: x[1], reverse=True)
# Mostrar las primeras 20 oraciones más similares
for i, (sentence, similarity) in enumerate(similarity_list[:20]):
    print(sentence)
    # print(f"Oración {i+1}:")
    # print("Texto:", sentence)
    # print("Similitud:", similarity)
    # print()

Permite crear nuevos Estados que permitan identificar fases de la evolución de los artículos, como por ejemplo, los artículos obsoletos.

Esto hace que un artículo pueda tener una estructura que deberá definir, así como un alcance que deberá indicar: ¿controlo o no su stock?, ¿es un artículo comercializable?
¿Cómo incorporar los Artículos al sistema?
Existen dos formas de dar de alta los artículos al sistema: con la migración de los datos mediante la importación de ficheros o con el registro manual directamente en el sistema, ingresando los datos de nuestros artículos uno a uno.

Tipos de Artículos: en la que configurará la relación entre los tipos-familias y las características que se van a manejar.

, ¿es un kit?, etc…
Antes de iniciar el registro de cualquier artículo debemos analizar cuáles son los grupos o Tipos de Artículos en los que vamos a clasificar nuestras referencias.
El primer paso del registro de las referencias, artículos o productos con los que vamos a operar, consiste

In [32]:
from transformers import pipeline

# Cargar el modelo de Q&A en español
qa_pipeline  = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
    )
)
qa_pipeline.tokenizer.truncate_sequences = 100

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Cargar el modelo GPT-2 en español
model_name = pipeline("text2text-generation",model="datificate/gpt2-small-spanish")
# tokenizer2 = AutoTokenizer.from_pretrained(model_name)
# model2 = AutoModelForCausalLM.from_pretrained(model_name)


The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [26]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")
# print(qa_pipeline.predict(tokenizer.encode("Hola mundo")))

In [35]:
# Texto completo (tu variable "text" cargada anteriormente)
document = """Permite crear nuevos Estados que permitan identificar fases de la evolución de los artículos, como por ejemplo, los artículos obsoletos.

Esto hace que un artículo pueda tener una estructura que deberá definir, así como un alcance que deberá indicar: ¿controlo o no su stock?, ¿es un artículo comercializable?
¿Cómo incorporar los Artículos al sistema?
Existen dos formas de dar de alta los artículos al sistema: con la migración de los datos mediante la importación de ficheros o con el registro manual directamente en el sistema, ingresando los datos de nuestros artículos uno a uno.

Tipos de Artículos: en la que configurará la relación entre los tipos-familias y las características que se van a manejar.

, ¿es un kit?, etc…
Antes de iniciar el registro de cualquier artículo debemos analizar cuáles son los grupos o Tipos de Artículos en los que vamos a clasificar nuestras referencias.
El primer paso del registro de las referencias, artículos o productos con los que vamos a operar, consiste en establecer los Tipos de Artículos.

Por un lado nos permitirá saber qué artículos de mi base de datos se encuentran activos en mis actividades y por otra mantendrá un histórico sobre los artículos obsoletos determinantes para realizar valoraciones en almacenes de material remanente.

Los artículos son una de las bases principales para el uso de la aplicación, por ello se denominan “Maestro de Información”.
Puedes desactivar las opciones de los conceptos que no interese trasladar a las fichas de los artículos ‘hijo’.

Codificar automáticamente los artículos a partir de los identificadores de Tipo y Familia de artículo.
Un ejemplo de la particularidad de gestión de los artículos reside en los aspectos que requiere para realizar correctamente las funciones esperadas.
Artículos Activos y sin Uso
Permite localizar los articulos que no han tenido movimientos desde una fecha determinada con la finalidad de saber en todo momento qué articulos tenemos y no usamos.

Inicia el registro de los Artículos.
La clasificación de artículos consiste en establecer una tipificación para nuestras referencias o artículos, proceso que se realiza en Tipos de Artículos.

Compras: pertenezca a un tipo de artículo con la característica “compra” + que el estado del artículo esté activo.

Generar referencias ‘hijo’:
Una vez registrados los artículos procede a generar los nuevos artículos según sus características, para ello:
Acceda a la ficha del artículo ‘padre’.

Como último paso, el sistema muestra el cuadro de diálogo ‘Generar Artículos nuevos’ donde se muestra agrupada (según solapas de datos) la información de la ficha del artículo ‘padre’: unidades, clientes, proveedores, etc.
Veamos a continuación de forma breve, cómo el estado o el bloqueo de artículos ponen de manifiesto la necesidad de realizar un correcto análisis de la información requerida durante el proceso de registro del maestro de artículos.

Efectos:
Se propone el Código y/o la Descripción del artículo con los datos que hayamos configurado en el criterio de codificación: tipo, familia, la unidad, empresa, marca, etc.
Durante los procesos de gestión en los que interviene el artículo, el sistema le permite visualizar la descripción del artículo en el idioma del cliente  / proveedor, dato que se muestra en los campos de referencias.
Tendremos especial cuidado al realizar los registros de los artículos ya que de ello depende el correcto funcionamiento del sistema."""

In [44]:
# generation = f"answer: ¿Qué son los artículos? context: {document}"
# input_ids = tokenizer2.encode(generation, return_tensors="pt")
# output = model2.generate(input_ids, max_length=150)
model_name(f"answer: ¿Qué son los artículos? context: {document}")

c:\Users\a.obregon\.conda\envs\kMeans\Lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\a.obregon\.conda\envs\kMeans\Lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 716, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_ne

[{'generated_text': 'answer: ¿Qué son los artículos? context: Permite crear nuevos Estados que permitan identificar fases de la evolución de los artículos, como por ejemplo, los artículos obsoletos.\n\nEsto hace que un artículo pueda tener una estructura que deberá definir, así como un alcance que deberá indicar: ¿controlo o no su stock?, ¿es un artículo comercializable?\n¿Cómo incorporar los Artículos al sistema?\nExisten dos formas de dar de alta los artículos al sistema: con la migración de los datos mediante la importación de ficheros o con el registro manual directamente en el sistema, ingresando los datos de nuestros artículos uno a uno.\n\nTipos de Artículos: en la que configurará la relación entre los tipos-familias y las características que se van a manejar.\n\n, ¿es un kit?, etc…\nAntes de iniciar el registro de cualquier artículo debemos analizar cuáles son los grupos o Tipos de Artículos en los que vamos a clasificar nuestras referencias.\nEl primer paso del registro de las

In [42]:
generated_text = tokenizer2.decode(output[0], skip_special_tokens=True)
print("Respuesta generada:", generated_text)

Respuesta generada: answer: ¿Qué son los artículos? context: Permite crear nuevos Estados que permitan identificar fases de la evolución de los artículos, como por ejemplo, los artículos obsoletos.

Esto hace que un artículo pueda tener una estructura que deberá definir, así como un alcance que deberá indicar: ¿controlo o no su stock?, ¿es un artículo comercializable?
¿Cómo incorporar los Artículos al sistema?
Existen dos formas de dar de alta los artículos al sistema: con la migración de los datos mediante la importación de ficheros o con el registro manual directamente en el sistema, ingresando los datos de nuestros artículos uno a uno.

Tipos de Artículos: en la que configurará la relación entre los tipos-familias y las características que se van a manejar.

, ¿es un kit?, etc…
Antes de iniciar el registro de cualquier artículo debemos analizar cuáles son los grupos o Tipos de Artículos en los que vamos a clasificar nuestras referencias.
El primer paso del registro de las referencia

In [33]:


# Preguntas de ejemplo
questions = "¿Qué son los artículos?"
print("Pregunta:", questions)
result = qa_pipeline(question=questions, context=document)
print(result)
print("Respuesta:", result["answer"])

# for question in questions:
#     print("Score:", result["score"])
#     print()

Pregunta: ¿Qué son los artículos?
{'score': 0.46201518177986145, 'start': 1307, 'end': 1364, 'answer': 'una de las bases principales para el uso de la aplicación'}
Respuesta: una de las bases principales para el uso de la aplicación
